# Fundamentals of Software Systems: Software Evolution, Part I - Assignment

In [1]:
from pydriller import Repository, Git

## Task 2: Complexity hotspots

To checkout the last major release 3.6.0, the following git commands were used: 
1. git clone https://github.com/apache/kafka.git
2. git checkout tags/3.6.0 -b 3.6  

In [49]:
# Create a list (pandas dataframe) of all .java files
import glob
import pandas as pd


def df_to_csv(df):
    df.to_csv('java_files.csv', index=False)


def csv_to_df():
    return pd.read_csv('java_files.csv')


java_files = []
for filename in glob.glob('./kafka/**/*.java', recursive=True):
    java_files.append(filename)

df = pd.DataFrame()
df['filename'] = java_files
df_to_csv(df)

In [50]:
# calculate cyclomatic complexity / LOC for each file using lizard
import lizard

df = csv_to_df()
cc_list = []
nloc_list = []
for i in df.index:
    cc = None
    nloc = None
    try:
        i = lizard.analyze_file(df.iloc[i]['filename'])
        cc = i.function_list[0].__dict__['cyclomatic_complexity']
        nloc = i.function_list[0].__dict__['nloc']
    except:
        pass
    cc_list.append(cc)
    nloc_list.append(nloc)

df['cc'] = cc_list
df['nloc'] = nloc_list
df_to_csv(df)